<a href="https://colab.research.google.com/github/robdicoco/solid-robot/blob/main/Google_agente_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Google GenAI library
%pip -q install google-genai



In [2]:
# Configure the Google Gemini API Key

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('Gemini_API')

In [3]:
# Configure the Gemini SDK client

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Ask Gemini for information more recent than its knowledge base

from IPython.display import HTML, Markdown

#  Ask the model about the next AI immersion ###
response = client.models.generate_content(
    model=MODEL_ID,
    contents='When will be the next imersion of Google Gemini from Alura?'
)


# Display the answer on the screen
display(Markdown(f"Response :\n {response.text}"))

Response :
 I understand you're asking about the next "Imersão" (Immersion) event focused on Google Gemini from Alura.

To find the exact dates for the next Imersão Google Gemini from Alura, it's best to check the following resources:

*   **Alura's official website:** Go to [alura.com.br](https://www.alura.com.br) and look for upcoming events, courses, or "Imersão" programs.
*   **Alura's social media channels:** Follow Alura on platforms like YouTube, Instagram, Twitter, and LinkedIn. They often announce events and new courses there.
*   **Alura's email newsletter:** If you're subscribed to Alura's newsletter, you'll likely receive updates about upcoming Imersões.

By checking these sources, you'll get the most accurate and up-to-date information about Alura's events related to Google Gemini.

In [5]:
# Ask Gemini for information using Google Search as context


response = client.models.generate_content(
    model=MODEL_ID,
    contents='When will be the next imersion of Google Gemini from Alura?',
    config = {"tools": [{"google_search": {}}]}
)

# Display the response
display(Markdown(f"Response:\n {response.text}"))

Response:
 The next Alura and Google's "AI Immersion" with Google Gemini is scheduled to take place between May 12 and May 16, 2025. Registration is open until May 11, 2025. This initiative is 100% free and offers a certificate of participation upon completion.


In [6]:
# Display the search query
print(f"Search performed: {response.candidates[0].grounding_metadata.web_search_queries}")
# Display the URLs used as basis for the response
print(f"Pages used in the response: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Search performed: ['Alura Google Gemini immersion dates']
Pages used in the response: alura.com.br, clickpetroleoegas.com.br, clickpetroleoegas.com.br



In [7]:
# Install Google ADK Agents Framework #
%pip install -q google-adk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [8]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # To create content (Content and Part)
from datetime import date
import textwrap # To better format text output
from IPython.display import display, Markdown  # To make HTTP requests
import warnings

warnings.filterwarnings("ignore")

In [9]:
# Helper function that sends a message to an agent via Runner and returns the final response
def call_agent(agent: Agent, message_text: str) -> str:
    # Creates an in-memory session service
    session_service = InMemorySessionService()
    # Creates a new session (you can customize IDs as needed)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Creates a Runner for the agent
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Creates the input message content
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Asynchronously iterates through the events returned during agent execution
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [10]:
# Helper function to display formatted text in Markdown in Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [11]:
##########################################
# --- Agent 1: News Finder           --- #
##########################################
def agent_news_finder(topic, today_date):
    news_finder = Agent(
        name="agent_news_finder",
        model="gemini-2.0-flash",
        instruction="""

        """,
        description="Agent that searches for news on a given topic.",
        tools=[google_search]
    )

    input_agent_news_finder = f"Topic: {topic}\nCurrent date: {today_date}"
    # Execute the agent
    news_found = call_agent(news_finder, input_agent_news_finder)
    return news_found



In [ ]:
# Helper function that sends a message to an agent via Runner and returns the final response
def call_agent(agent: Agent, message_text: str) -> str:
    # Creates an in-memory session service
    session_service = InMemorySessionService()
    # Creates a new session (you can customize IDs as needed)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Creates a Runner for the agent
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Creates the input message content
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Asynchronously iterates through the events returned during agent execution
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Helper function to display formatted text in Markdown in Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [16]:
##########################################
# --- Agent 1: News Finder           --- #
##########################################
def agent_news_finder(topic, today_date):
    news_finder = Agent(
        name="agent_news_finder",
        model="gemini-2.0-flash",
        instruction="""
          You are a news research expert.
          Your task is to find up to three of the most recent and highly relevant news articles on the given topic using (google_search).
          Focus on finding reputable news sources published within the last week.
          If the initial topic yields few results, try one closely related alternative.
          Present the findings as a bulleted list with the title and a brief summary of each article.
        """,
        description="Agent that searches for news on a given topic.",
        tools=[google_search]
    )

    input_agent_news_finder = f"Topic: {topic}\nCurrent date: {today_date}"
    # Execute the agent
    news_found = call_agent(news_finder, input_agent_news_finder)
    return news_found



In [12]:
################################################
# --- Agent 2: Post Planner                --- #
################################################
def agent_post_planner(topic, news_found):
    agent_planner = Agent(
        name="agent_post_planner",
        model="gemini-2.0-flash",
        instruction="""
            You are a post planner expert in social network.
            Your task is to create a post plan for Instagram based on the news found.
            You can use (google_search) to cleate a plan about the most important topics.
            You can use (google_search) for further refine the post on the most relevant tópics
           in the end, you will choose the most relevant topic among all of them, based on your
           research and return to this topic, its most relevant points, and the plan with the
           subjects to be addressed in the post that will be written there later.
        """,
        description="Agent that plans post",
        tools=[google_search]
    )


    input_agent_planner = f"Topic: {topic}\nNews found: {news_found}"
    # Execute the agent
    post_plan = call_agent(agent_planner, input_agent_planner)
    return post_plan


In [13]:

######################################
# --- Agent 3: Post Writer       --- #
######################################
def agent_writer(topic, post_plan):
    writer = Agent(
        name="agent_writer",
        model="gemini-2.0-flash",
        instruction="""
            You are a Creative Writer specializing in creating viral posts for social media.
            You write posts for RobDC a Web3 developer, with MSc in Biomedical Engineering, BSc. in Systems Analysis, Mechatronics Tech.
            Use the theme provided in the post plan and the most relevant points provided and, based on that,
            write a draft Instagram post on the indicated topic.
            The post should be engaging, informative, use simple language, and include 2 to 5 hashtags at the end.
            """,
        description="Agent writer of engaging posts for Instagram"
    )

    input_agent_writer = f"Topic: {topic}\nPost plan: {post_plan}"
    # Execute the agent
    draft = call_agent(writer, input_agent_writer)
    return draft


In [14]:
##########################################
# --- Agent 4:     Quality Reviewer --- #
##########################################
def agent_reviewer(topic, draft_information):
    reviewer = Agent(
        name="agent_reviewer",
        model="gemini-2.0-flash",
        instruction="""
            You are a meticulous Content Editor and Reviewer, specializing in social media posts, with a focus on Instagram.
            Since you have a young audience, between 18 and 30 years old, use an appropriate writing tone.
            Review the Instagram post draft below on the indicated topic, checking for clarity, conciseness, correctness, and tone.
            If the draft is good, reply only 'The draft is great and ready to publish!'.
            If there are problems, point them out and suggest improvements.
            """,
        description="Agent reviewer of social media posts"
    )

    input_agent_reviewer = f"Topic: {topic}\nDraft: {draft_information}"
    # Execute the agent
    reviewed_text = call_agent(reviewer, input_agent_reviewer)
    return reviewed_text


In [17]:
today_date = date.today().strftime("%d/%m/%Y")

print("🚀 Starting the Instagram Post Creation System with 4 Agents 🚀🚀🚀")

# --- Get Topic from User ---
topic = input("❓ Please enter the TOPIC you want to create the trend post about: ")

# Insert agent system logic ################################################

if not topic:
    print("❌ Topic cannot be empty. Please try again.")
else:
    print(f"🔍 Searching for news on the topic: {topic}...")
    news_found = agent_news_finder(topic, today_date)
    print("📝 News found:\n", to_markdown(news_found))

    # print("📋 Creating post plan...")
    # post_plan = agent_post_planner(topic, news_found)
    # print("📋 Post plan:\n", to_markdown(post_plan))

    # print("✍️ Writing post draft...")
    # draft = agent_writer(topic, post_plan)

    # print("🔍 Reviewing post draft...")
    # reviewed_text = agent_reviewer(topic, draft)
    # print("🔍 Reviewed post:\n", to_markdown(reviewed_text))


🚀 Starting the Instagram Post Creation System with 4 Agents 🚀🚀🚀
❓ Please enter the TOPIC you want to create the trend post about: batata
🔍 Searching for news on the topic: batata...
📝 News found:
 <IPython.core.display.Markdown object>
